In [2]:
import urllib.request
import urllib.parse
import numpy as np
import pandas as pd
import json
import sqlite3
import pprint as pp

In [3]:
con = sqlite3.connect("database")
cur = con.cursor()
cur.execute("SELECT * FROM movie_titles_cleaned")
res = cur.fetchall()
df = pd.DataFrame(res, columns=["id", "year", "country", "title", "title_cleaned", "title_bracket", "title_bracket_2", "list_id"])
df = df.set_index(["id"])
df

,year,country,title,title_cleaned,title_bracket,title_bracket_2,list_id
id,,,,,,,
1,1950,Hindi,Sangram,Sangram,None,None,1
2,1950,Hindi,Raj Mukut,Raj Mukut,None,None,1
3,1950,Hindi,Beqasoor,Beqasoor,None,None,1
4,1950,Hindi,Hamara Ghar,Hamara Ghar,None,None,1
5,1950,Hindi,Gulnar,Gulnar,None,None,1
...,...,...,...,...,...,...,...
103643,1979,Turkish,Sona Kalan Dona Kalır,Sona Kalan Dona Kalır,None,None,684
103644,1979,Turkish,Bir Çiçek Üç Böcek,Bir Çiçek Üç Böcek,None,None,684
103645,1979,Turkish,Maceranın Böylesi,Maceranın Böylesi,None,None,684


In [4]:
print(df.loc[1])

year                  1950
country              Hindi
title              Sangram
title_cleaned      Sangram
title_bracket         None
title_bracket_2       None
list_id                  1
Name: 1, dtype: object


In [7]:
df_len = len(df)
collector = []
count = 31486
dump = 0
failed = []
log = []
need = []
for i in range(96300, df_len):
    dump += 1
    entry = [i]
    try:
        for title in ['title', 'title_cleaned', 'title_bracket', 'title_bracket_2']:
            if df.loc[i][title]:
                url = ("http://www.omdbapi.com/?i=tt3896198&apikey=ab3d2a11" +
                       "&t=" + urllib.parse.quote(str(df.loc[i][title])).replace(" ", "+") +
                       "&y=" + str(df.loc[i]['year']) +
                       "&plot=full")
                data_raw = urllib.request.urlopen(url)
                data = json.loads(data_raw.read())
                if data["Response"] == 'True':
                    collector.append(data)
                    entry.append("success")
                    while len(entry) < 5:
                        entry.append("")
                    count += 1
                    break
                if data["Response"] == 'False':
                    entry.append("failed")
            else:
                entry.append("failed")
    except:
        count += 1
        entry = [i, "ERROR", "ERROR", "ERROR", "ERROR"]
    # print log
    print(entry)
    log.append(entry)
    # dump data
    if dump == 100:
        try:
            with open("data.json", "a") as fh:
                json.dump(collector, fh, indent=2, ensure_ascii=False)
            print(f"data dumped: total {count}/{i}. ")
        except:
            need.append(collector)
            print(f"data dumped: failed.")
        collector = []
        dump = 0

[96300, 'failed', 'failed', 'failed', 'failed']
[96301, 'success', '', '', '']
[96302, 'failed', 'failed', 'failed', 'failed']
[96303, 'success', '', '', '']
[96304, 'success', '', '', '']
[96305, 'success', '', '', '']
[96306, 'success', '', '', '']
[96307, 'success', '', '', '']
[96308, 'success', '', '', '']
[96309, 'success', '', '', '']
[96310, 'success', '', '', '']
[96311, 'success', '', '', '']
[96312, 'failed', 'failed', 'failed', 'failed']
[96313, 'failed', 'failed', 'failed', 'failed']
[96314, 'success', '', '', '']
[96315, 'success', '', '', '']
[96316, 'success', '', '', '']
[96317, 'success', '', '', '']
[96318, 'success', '', '', '']
[96319, 'failed', 'failed', 'failed', 'failed']
[96320, 'success', '', '', '']
[96321, 'success', '', '', '']
[96322, 'failed', 'failed', 'failed', 'failed']
[96323, 'success', '', '', '']
[96324, 'success', '', '', '']
[96325, 'success', '', '', '']
[96326, 'failed', 'failed', 'failed', 'failed']
[96327, 'failed', 'failed', 'failed', 'faile

In [6]:
need = [collector]
# 69200 - 69299
# 96200 - 96299
temp = pd.DataFrame(log)
temp.to_csv("log.csv", index=False)